we import some clmm modules

In [ ]:
import sys
import os
import clmm

import clmm.dataops
from clmm.dataops import compute_tangential_and_cross_components, make_radial_profile, make_bins
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
from clmm import Cosmology
from clmm.support import mock_data as mock
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
import numpy as np
cosmo_clmm = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from numpy import random
import chainconsumer
import scipy

clmm.__version__

we import `clmm` module's core.

modules

In [ ]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from scipy.optimize import curve_fit

we define `astropy` and `ccl` cosmology object

In [ ]:
mock_cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

In [ ]:
moo = clmm.Modeling(massdef = 'critical', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(mock_cosmo)

we define the `Modeling` object `moo` to model the galaxy cluster.

we now define the type of desired profile : tangential reduced shear of Excess surface density for the stacking procedure by modifying the atribute. We use the method `Type` from the class to select DeltaSigma of reduced tangential shear.

In [ ]:
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]

# Estimator of the stacked excess surface density

- The maximum likelihood estimator of the excess surface density in the radial bin $[R, R + \Delta R[$ is defined by

$$\widehat{\Delta\Sigma}_+(R) = \frac{1}{\sum\limits_{l,s = 1} w_{ls}}
     \sum\limits_{l,s= 1}w_{ls}\langle\Sigma_{{\rm crit}}(z_s, z_l)^{-1}\rangle^{-1}\epsilon_+^{s},$$ where $\epsilon_+^{s}$ is the tangential ellipticity of the background galaxy with index $s$ (as source) relative to the dark matter halo position with index $l$ (as lens).
     
- The critical surface mass density expresses as $\Sigma_{{\rm crit}}(z_s, z_l) = \frac{c^2}{4 \pi G} \frac{D_A(z_s)}{D_A(z_l) D_A(z_s, z_l)}$, where $D_A(z_l), D_A(z_s)$ are respectively the angular diameter distance to the lens and to the source in physical units, and $D_A(z_s, z_l)$ is the angular diameter distance between the lens and the source.


- The average $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle$ is defined as $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle = \int_{z_l + \delta}^{+\infty} d z_s\ p_{\rm photoz,s}(z_s)\ \Sigma_{\rm crit}(z_s,z_l)^{-1}$, where $p_{\rm photoz,s}$ is the photometric probability density function for the background galaxy with index $s$.

This equation can be re-written as

$$\widehat{\Delta\Sigma}_+(R) = \frac{1}{\sum\limits_{l = 1} W_{l}}
     \sum\limits_{l,s= 1}W_{l}\Delta\Sigma_l(R)$$
     
where $$\Delta\Sigma_l(R) = \frac{1}{\sum\limits_{s = 1} w_{ls}}
     \sum\limits_{s= 1}w_{ls}\langle\Sigma_{{\rm crit}}(z_s, z_l)^{-1}\rangle^{-1}\epsilon_+^{s}$$
     
and $$W_{l} = \sum\limits_{s= 1}w_{ls}$$

### Weights $w_s$

The quantities $w_{s}$ are the weights that maximise the sigmnal-to-noise ratio of the excess surface density estimator. They downweight the galaxies that are close in redshift to the cluster (where the lensing signal is weak). They include the lack of informations on both redshift and shape reconstruction for each background galaxies. 

- In the case where there is no error on the shape measurement (for the purpose of cosmoDC2 galaxies), the weight writes $w_{ls} = \langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle^{2}$,

- In the case of true redshift $z_s$, the PDF reduces to a Dirac function centered at $z_s$, giving the average $\langle\Sigma_{\rm crit}(z_l,z_s)^{-1}\rangle = \Sigma_{\rm crit}(z_l,z_s)^{-1} $ and the weight $w_{s} = \Sigma_{\rm crit}(z_s,z_l)^{-2}$,

We now define some redshift interval and input mass interval to select GalaxyCluster object in preselected galaxy catalogs from `cosmoDC2`

# Dark matter halo catalog for the stack

In [ ]:
z_bin = [0.2,0.25]
logm_bin = [14, 14.1] #Solar Mass
logm_bin = np.array(logm_bin)
n_catalogs = 50
ngals = 10000

we define a set of selected cluster with given true masses ans true redshifts

In [ ]:
cluster_m = 10**((logm_bin[1] - logm_bin[0])*np.random.random(n_catalogs) + logm_bin[0]) #in M_sun
cluster_z = (z_bin[1] - z_bin[0])*np.random.random(n_catalogs) + z_bin[0]
lnc = abs(np.log(4) + 0.1*np.random.randn(n_catalogs))
concentration = np.exp(lnc)

In [ ]:
plt.scatter(cluster_m, concentration)
plt.xlabel(r'$M_{\rm 200c}$', fontsize = 20)
plt.ylabel(r'$c_{\rm 200c}$', fontsize = 20)

We add each galaxy catalogs that correponds to binning criteria to the `shear` object through the `Add(self,'file_name')` method.

In [ ]:
shapenoise = 0.005

In [ ]:
def excess_surface_density(single_catalog = None, radial_bin = None, sigma_c = None):
    r"""
    Attributes:
    -----------
    
    single_catalog : GalaxyCluster object
    
    radial_bin : liste
        radial bins to evaluate the binned excess surface density
    
    sigma_c : string 
        column name in the single_catalog for the critical surface mass density
        
    Returns:
    --------
    
    ds, r, sum_weights : array, array, array
        the binned excess surface density, the binned radius, the sum of weights w_ls in each radial bin

    """
    
    ds = np.zeros(len(radial_bin))
    
    r = np.zeros(len(radial_bin))
    
    sum_weights = np.zeros(len(radial_bin))
    
    for i, r_bin in enumerate(radial_bin):

        mask = (single_catalog.galcat['R'] > r_bin[0])*(single_catalog.galcat['R'] < r_bin[1])
        
        w_ls = 1./single_catalog.galcat[sigma_c][mask]**2
        
        ds[i] = np.average(single_catalog.galcat['et'][mask]*single_catalog.galcat[sigma_c][mask], weights = w_ls)
        
        r[i] = np.average(single_catalog.galcat['R'][mask])
        
        sum_weights[i] = np.sum( w_ls )
        
    return ds, r, sum_weights

## Generating background galaxy catalogs & estimating individual shear profile

We use `clmm` to generate `n_catalogs` background galaxy catalogs for each clusters in the sample. We estimate the individual shear profile and store the individual data:

- `ds_single`, `r_single` the single excess surface density extimated in the radial bins, and the average radius in each radial bins
- `W_l` the sum of the weight `w_ls` for each lens-source pairs in each radial bins.
We difine two possible weights `w_ls`, first using true redshifts of sources, second using the photometric redshift pdf for each sources as presented above. In the function `excess_surface_density`, the argument `sigma_c` will be computed using the true redshift of galaxy, of it will be given by $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle^{-1}$.

In [ ]:
use_photoz_weights = True

In [ ]:
%time
GalaxyCluster = []

for i in range(n_catalogs):
    
    if i%10 == 0: print(i)
    
    noisy_data_z = mock.generate_galaxy_catalog(cluster_m[i], cluster_z[i], concentration[i], 
                                                cosmo, 
                                                zsrc = 'chang13', 
                                                Delta_SO=200, 
                                                massdef='critical',
                                                halo_profile_model='nfw', 
                                                zsrc_min=cluster_z[i] + 0.1,
                                                zsrc_max=3., 
                                                field_size=10., 
                                                shapenoise=shapenoise, 
                                                photoz_sigma_unscaled=0.05, 
                                                ngals=ngals)
    
    if use_photoz_weights == True:
        
        pzbins_constant = np.linspace(0,3,30)
        
        sigma_crit_1 = 1./mock_cosmo.eval_sigma_crit(cluster_z[i], pzbins_constant)

        photoz_matrix = np.zeros([len(noisy_data_z), len(pzbins_constant)])
        
        r"""
        I added this line since the photoz pdf is not computed on a constant redshift grid. simps integral can be optimized using the same x-axis for multiple integrals.
        =================================================================================================================================================================
        """

        for f in range(len(noisy_data_z['id'])):

            photoz_matrix[f,:] = np.interp(pzbins_constant, noisy_data_z['pzbins'][f], noisy_data_z['pzpdf'][f])
            
        r"""
        =================================================================================================================================================================
        """

        unormed_integral = scipy.integrate.simps(photoz_matrix * sigma_crit_1, x = pzbins_constant, axis = 1)

        norm = scipy.integrate.simps(photoz_matrix, x = pzbins_constant, axis = 1)

        noisy_data_z['sigma_c_photoz'] = (unormed_integral/norm)**(-1)
        
    
    cl = clmm.GalaxyCluster('mock_cluster', 0, 0, cluster_z[i], noisy_data_z)
    
    cl.compute_tangential_and_cross_components(geometry="flat",
                                              shape_component1='e1', shape_component2='e2', 
                                              tan_component='et', cross_component='ex',
                                              add=True, 
                                              is_deltasigma = False, 
                                              cosmo = cosmo)
    
    cl.galcat['R'] = mock_cosmo.eval_da_z1z2(0,cluster_z[i])*cl.galcat['theta']
    
    ang, phi = clmm.dataops._compute_lensing_angles_flatsky(cl.ra, cl.dec, cl.galcat['ra'], cl.galcat['dec'])

    cl.galcat['phi'] = np.array(phi)
    
    cl.galcat['sigma_c_true'] = mock_cosmo.eval_sigma_crit(cluster_z[i], cl.galcat['ztrue'])
    
    GalaxyCluster.append(cl)

## Estimating single excess surface density profiles

$$\widehat{\Delta\Sigma}_+(R) = \frac{1}{\sum\limits_{l = 1} W_{l}}
     \sum\limits_{l,s= 1}W_{l}\Delta\Sigma_l(R)$$

In [ ]:
new_bins = np.logspace(np.log10(0.4), np.log10(5), 20)
radial_bin = [[new_bins[s],new_bins[s+1]] for s in range(len(new_bins)-1)]

In [ ]:
names = ['ds_single', 'r_single', 'W_l','c', 'z', 'mass']

Stack_file_true = {name : [] for name in names}

Stack_file_photoz = {name : [] for name in names}

for i in range(n_catalogs):
    
    ds_single, r_single, sum_weights_single = excess_surface_density(single_catalog = GalaxyCluster[i], radial_bin = radial_bin, sigma_c = 'sigma_c_true')
    
    data_to_store = [ds_single, r_single, sum_weights_single, concentration[i], cluster_z[i], cluster_m[i]]
    
    for d, name in enumerate(names):
        
        Stack_file_true[name].append(data_to_store[d])
        
    ds_single, r_single, sum_weights_single = excess_surface_density(single_catalog = GalaxyCluster[i], radial_bin = radial_bin, sigma_c = 'sigma_c_photoz')
    
    data_to_store = [ds_single, r_single, sum_weights_single, concentration[i], cluster_z[i], cluster_m[i]]
    
    for d, name in enumerate(names):
        
        Stack_file_photoz[name].append(data_to_store[d])

# Stacked excess surface density profile : true z and photoz

## Stacked profile

We estimate the mean excess surface density profile `ds_mean` and the mean radius `r_mean`

In [ ]:
#stacked excess surface density profile - true redshift case

ds_mean_true = np.average(Stack_file_true['ds_single'], weights = Stack_file_true['W_l'], axis = 0)
r_mean_true = np.average(Stack_file_true['r_single'], weights = None, axis = 0)

#stacked excess surface density profile - photoz redshift case
ds_mean_photoz = np.average(Stack_file_photoz['ds_single'], weights = Stack_file_photoz['W_l'], axis = 0)
r_mean_photoz = np.average(Stack_file_photoz['r_single'], weights = None, axis = 0)

## Covariance matrix

We compute the sample covariance matrix defined as
$$
    (\mathbf{C}^{\rm sample})_{i,j} = \frac{1}{N_{\rm cluster} - 1}\sum_{k = 0}^{N_{\rm cluster}}[\Delta\Sigma^k(R_i) - \overline{\Delta\Sigma}(R_i)] [\Delta\Sigma^k(R_j) - \overline{\Delta\Sigma}(R_j)],
$$
and $\overline{\Delta\Sigma}(R_i)$ is denoted by
$$
    \overline{\Delta\Sigma}(R_i) = \frac{1}{N_{\rm cluster}}\sum_{k = 0}^{N_{\rm boot}} \Delta\Sigma^k(R_i).
$$

In [ ]:
#covariance matrix
sample_covariance_stack_true = np.cov(np.array(Stack_file_true['ds_single']).T)/n_catalogs
sample_covariance_stack_photoz = np.cov(np.array(Stack_file_photoz['ds_single']).T)/n_catalogs

# Single cluster : Generating background galaxy catalogs

To make standard comparison with the stack analysis, we use also consider a single cluster

In [ ]:
noisy_data_z_single = mock.generate_galaxy_catalog(np.mean(cluster_m), np.mean(cluster_z), np.mean(concentration), cosmo, 
                                                zsrc = 'chang13', 
                                                Delta_SO=200, 
                                               massdef='critical',
                                               halo_profile_model='nfw', zsrc_min=np.mean(cluster_z) + 0.1,
                           zsrc_max=3., field_size=10., shapenoise=shapenoise, photoz_sigma_unscaled=0, ngals=ngals)

cl_single = clmm.GalaxyCluster('mock_cluster', 0., 0., np.mean(cluster_z), noisy_data_z_single)

cl_single.compute_tangential_and_cross_components(geometry="flat",
                                                  shape_component1='e1', shape_component2='e2', 
                                                  tan_component='et', cross_component='ex',
                                                  add=True, is_deltasigma = False, cosmo = cosmo)

cl_single.galcat['R'] = mock_cosmo.eval_da_z1z2(0,np.mean(cluster_z))*cl_single.galcat['theta']

ang, phi = clmm.dataops._compute_lensing_angles_flatsky(cl_single.ra, cl_single.dec, cl_single.galcat['ra'], cl_single.galcat['dec'])

cl_single.galcat['phi'] = np.array(phi)

cl_single.galcat['sigma_c'] = mock_cosmo.eval_sigma_crit(np.mean(cluster_z), cl_single.galcat['z'])

# Single cluster : excess surface density

## Excess surface density profile

In [ ]:
#excess surface density profile
ds_single,r_single,sum_weight_single = excess_surface_density(single_catalog = cl_single, radial_bin = radial_bin, sigma_c = 'sigma_c')

## Covariance matrix

We estimate the single excess surface density covariance matrix by the delete-1 Jackknife method : We split the sky area arround the cluster center in $N_{\rm JK}$ regions, and we exclude one region at a time and re-estimate the excess surface density profile:, the covariance writes

$$
    (\mathbf{C}^{\rm JK})_{i,j} = \frac{N_{\rm JK} - 1}{N_{\rm JK}}\sum_{k = 0}^{N_{\rm JK}}[\Delta\Sigma^k(R_i) - \overline{\Delta\Sigma}(R_i)] [\Delta\Sigma^k(R_j) - \overline{\Delta\Sigma}(R_j)]
$$
with the mean 
$$
    \overline{\Delta\Sigma}(R_i) = \frac{1}{N_{\rm JK}}\sum_{k = 0}^{N_{\rm JK}} \Delta\Sigma^k(R_i).
$$

In [ ]:
def jackknife(single_catalog = None, N_jk = None):
    
    r"""
    Attributes:
    -----------
    single_catalog : GalaxyCluster object
        GalaxyCluster object of a single cluster 
    N_jk : int
        number of jackknife regions
        
    Returns:
    --------
    cov : array
        Jackknife delete-1 covariance matrix
    
    """
    
    phi = np.linspace(-np.pi, np.pi, N_jk + 1)
    
    phi_bin = binning(phi)
    
    gt = []
    
    mask_list_in = [(phi_[0] < single_catalog.galcat['phi'])*(single_catalog.galcat['phi'] < phi_[1]) for phi_ in phi_bin]
    
    for s, phi_ in enumerate(phi_bin):
        
        mask_in = mask_list_in[s]
        
        mask_out = np.invert(mask_in)
        
        plt.scatter(single_catalog.galcat['ra'][mask_in], single_catalog.galcat['dec'][mask_in], s = 1)
        
        data_cut = single_catalog.galcat[mask_out]
        
        cl_cut_jack = clmm.GalaxyCluster('Stack', single_catalog.ra, single_catalog.dec, single_catalog.z, single_catalog.galcat[mask_out])
        
        ds, r, sum_weights = excess_surface_density(single_catalog = cl_cut_jack, radial_bin = radial_bin, sigma_c = 'sigma_c')
        
        gt.append(ds)

    gt = np.array(gt)
    
    cov = np.cov(gt.T)*(N_jk-1)**2/N_jk
    
    plt.xlabel('ra')
    plt.ylabel('dec')
    plt.axis('equal')
        
    return cov

In [ ]:
#Jackknife covariance matrix
cov_single = jackknife(single_catalog = cl_single, N_jk = 100)

In [ ]:
fig, axs = plt.subplots(2,2, figsize = (20,10))
#axs[0,0].imshow(np.corrcoef(cov_ss), vmin = -1,, cmap = 'bwr', origin = 'lower')
min_value = min(sample_covariance_stack_true.flatten())
im = axs[0,0].imshow(cov_single, cmap = 'Reds', origin = 'lower', label = 'single-jackknife')
plt.colorbar(im, ax=axs[0,0])
axs[0,1].errorbar(r_single,ds_single, np.sqrt(cov_single.diagonal()), label = 'single')
moo.set_concentration(concentration.mean())
moo.set_mass(cluster_m.mean())
axs[0,1].plot(r_single,moo.eval_excess_surface_density(r_single, cluster_z.mean()), label = 'fiducial')
axs[0,1].set_ylim(0.3*1e13, 3.5*1e13)

im = axs[1,0].imshow(sample_covariance_stack_true, cmap = 'Reds', origin = 'lower' ,label = 'stack-sample')
plt.colorbar(im, ax=axs[1,0])

plt.figure(figsize = (20,10))
for i in range(n_catalogs):
    axs[1,1].plot(Stack_file_true['r_single'][i], Stack_file_true['ds_single'][i], alpha = 0.01, c = 'grey', linewidth =20)
axs[1,1].errorbar(r_mean_true,ds_mean_true, np.sqrt(sample_covariance_stack_true.diagonal()), linewidth = 3, label = 'stack - true z')
axs[1,1].errorbar(r_mean_photoz,ds_mean_photoz, np.sqrt(sample_covariance_stack_photoz.diagonal()), linewidth = 3,fmt = '.', label = 'stack - photoz')
axs[1,1].set_ylim(0.3*1e13, 3.5*1e13)

axs[0,0].set_title('covariance matrix', fontsize = 20)
axs[0,1].set_title('Excess surface density profile', fontsize = 20)
for i in range(2):
    axs[i,1].set_xlim(min(r_single), max(r_single))
    axs[i,1].set_ylim(0.3*1e13, 3.5*1e13)
    axs[i,1].set_xlabel('R [Mpc]', fontsize = 20)
    axs[i,1].set_ylabel(r'$\Delta\Sigma$', fontsize = 20)
    
for i in range(2):
    for j in range(2):
        axs[i,j].legend(fontsize = 20)

# Stack VS single excess surface density : Fitting cluster mass and concentration

We fit the cluster mass and concentration using the `emcee` package 

In [ ]:
def lnL(theta, r, ds, cov, z):
    
    logm, c = theta
    
    if c < 0.01: return -np.inf

    moo.set_mass(10**logm)
    
    moo.set_concentration(c)
    
    y_th = moo.eval_excess_surface_density(r,z)
    
    delta = (y_th - ds)
    
    inv_cov = np.linalg.inv(cov)
    
    lnL_ds = -0.5*np.sum(delta*inv_cov.dot(delta))
    
    return lnL_ds

In [ ]:
import emcee
initial = [np.log10(np.mean(cluster_m)),np.mean(concentration)]
npath = 500
nwalkers = 50
pos = initial + 0.01 * np.random.randn(nwalkers, len(initial))
nwalkers, ndim = pos.shape
sampler_single = emcee.EnsembleSampler(nwalkers, ndim, lnL, args = (r_single, ds_single, cov_single, np.mean(cluster_z)))
print('Single cluster : running...')
sampler_single.run_mcmc(pos, npath, progress=True);

In [ ]:
print('Stack true z : running...')
sampler_stack_true = emcee.EnsembleSampler(nwalkers, ndim, lnL, args = (r_mean_true, ds_mean_true, sample_covariance_stack_true, np.mean(cluster_z)))
sampler_stack_true.run_mcmc(pos, npath, progress=True);

In [ ]:
print('Stack photoz : running...')
sampler_stack_photoz = emcee.EnsembleSampler(nwalkers, ndim, lnL, args = (r_mean_photoz, ds_mean_photoz, sample_covariance_stack_photoz, np.mean(cluster_z)))
sampler_stack_photoz.run_mcmc(pos, npath, progress=True);

In [ ]:
flat_sample_stack_pre_cut = sampler_stack_true.get_chain(discard = 0, thin = 1, flat = True)
flat_sample_single_pre_cut = sampler_single.get_chain(discard = 0, thin = 1, flat = True)
plt.plot(flat_sample_single_pre_cut[:,0])
plt.plot(flat_sample_stack_pre_cut[:,0])
plt.ylabel(r'$\log_{10}(M_{200c})$', fontsize= 20)
plt.figure()
plt.plot(flat_sample_single_pre_cut[:,1])
plt.plot(flat_sample_stack_pre_cut[:,1])
plt.ylabel(r'$c_{200c}$', fontsize= 20)

In [ ]:
discard = 300
thin = 20

In [ ]:
flat_sample_stack_true = sampler_stack_true.get_chain(discard = discard, thin = thin, flat = True)
flat_sample_stack_photoz = sampler_stack_photoz.get_chain(discard = discard, thin = thin, flat = True)

In [ ]:
flat_sample_single = sampler_single.get_chain(discard = discard, thin = thin, flat = True)

In [ ]:
#flat_sample_stack_true.shape, flat_sample_single.shape

In [ ]:
labels = [r'$\log_{10} M_{\rm 200c}$',r'$c_{\rm 200c}$', ]

In [ ]:
import numpy as np
from chainconsumer import ChainConsumer

mean = [np.log10(np.mean(cluster_m)),np.mean(concentration)]
c = ChainConsumer()
c.add_chain(flat_sample_stack_true, parameters=labels, name = 'Stack - true z')
c.add_chain(flat_sample_stack_photoz, parameters=labels, name = 'Stack - photoz')
c.add_chain(flat_sample_single, parameters=labels, name = 'Single')
c.configure (kde = True,colors=['b', 'r','g'], shade=True, shade_alpha=0.2, bar_shade=True, label_font_size=11, sigma2d=False, sigmas = [1, 2], spacing = 0.0, tick_font_size=11, usetex=False)
plot_args = {}
plot_args['truth'] = mean
fig = c.plotter.plot(**plot_args)
fig.set_size_inches(10, 10)